In [1]:
import pandas as pd
pd.set_option('display.max_columns',None)

import sys
sys.path.append("../..")  # 상위 디렉토리를 sys.path에 추가

import requests
import pandas as pd
from mySetting import Client_ID, Client_Secret

# 네이버 API 키
client_id = Client_ID  # 클라이언트 ID
client_secret = Client_Secret  # 클라이언트 Secret

def get_coordinates(address):
    url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode'
    headers = {
        'X-NCP-APIGW-API-KEY-ID': client_id,
        'X-NCP-APIGW-API-KEY': client_secret,
    }
    params = {
        'query': address,
    }
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    
    if data['addresses']:
        x = data['addresses'][0]['x']
        y = data['addresses'][0]['y']
        return (x, y)
    else:
        return (None, None)

In [3]:
def get_traffic(new_bus_stop, new_station):
    # 버스정류장
    bus_stop = pd.read_csv('../../' + new_bus_stop, encoding='cp949')
    bus = bus_stop.loc[bus_stop['도시명'].str.contains('평택')][['정류장명','위도','경도']].rename(columns={'정류장명':'이름'})


    # 역
    station = pd.read_excel('../../' + new_station)
    train = station.loc[station['역사도로명주소'].str.contains('평택시')][['역사명','역사도로명주소','역위도','역경도']].rename(columns={'역사도로명주소':'주소','역위도':'위도','역경도':'경도','역사명':'이름'})

    return bus, train

In [4]:
new_bus_stop = 'new_data/교통시설/국토교통부_전국 버스정류장 위치정보_20231016.csv'
new_station = 'new_data/교통시설/전체_도시철도역사정보_20240630.xlsx'

bus, train = get_traffic(new_bus_stop, new_station)

In [5]:
bus.to_csv('../../preprocessed/교통시설/버스정류장.csv', index=False)
train.to_csv('../../preprocessed/교통시설/역.csv', index=False)